In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

In [129]:
n = 5 * 12 + 6
x = np.arange(n)
y = np.abs(np.random.randn(n))
y_cs = np.cumsum(y) * 1000

data = pd.DataFrame({"x": x, "y": y, "A": y_cs, "B": 2 * y_cs, "C": 3 * y_cs})

In [194]:
def sum_lines_traces(data, ycols, ycol_highlight, left, right, xcol):
    hover_template = (
        "<b>Month</b>: %{x}" + "<br><b>Cumulative Cost</b>: $%{y:,.0f}"
    )
    traces = []
    for column in ycols:
        if column == ycol_highlight:
            # plot from 0 to shade_l
            data_left = data.loc[data[xcol] < left]
            data_right = data.loc[data[xcol] > right]
            data_middle = data.loc[(data[xcol] >= left) & (data[xcol] <= right)]

            for d in [data_left, data_right, data_middle]:
                opacity = 1 if d is data_middle else 0.2
                traces.append(
                    go.Scatter(
                        x=d[xcol],
                        y=d[column],
                        mode="lines",
                        line=dict(color="rgba(65,40,200,1)"),
                        showlegend=False,
                        opacity=opacity,
                        hovertemplate=hover_template,
                        name=column,
                    )
                )

        else:
            traces.append(
                go.Scatter(
                    x=data[xcol],
                    y=data[column],
                    mode="lines",
                    line=dict(color="rgba(60,60,60,0.25)"),
                    showlegend=False,
                    hovertemplate=hover_template,
                    name=column,
                )
            )

    # add maker at left, right
    y_l = data.loc[data[xcol] == left, ycol_highlight].values[0]
    y_r = data.loc[data[xcol] == right, ycol_highlight].values[0]

    traces.append(
        go.Scatter(
            x=[left, right],
            y=[y_l, y_r],
            mode="markers",
            marker=dict(color="rgba(95,40,200,1)", size=6),
            showlegend=False,
            hoverinfo="skip",
        )
    )

    return traces

In [240]:
def vertical_traces(data, ycol_highlight, left, right, xcol):

    traces = []
    # plot from 0 to shade_l
    data_left = data.loc[data[xcol] < left, [ycol_highlight]]
    data_right = data.loc[data[xcol] > right, [ycol_highlight]]
    data_middle = data.loc[
        (data[xcol] >= left) & (data[xcol] <= right), [ycol_highlight]
    ]
    for d in [data_left, data_right, data_middle]:
        opacity = 1 if d is data_middle else 0.2
        traces.append(
            go.Scatter(
                x=[0] * len(d),
                y=d[ycol_highlight],
                mode="lines",
                line=dict(color="rgba(65,40,200,1)"),
                showlegend=False,
                opacity=opacity,
                hoverinfo="skip",
            )
        )

    # add marker at left, right
    y_l = data_middle[ycol_highlight].min()
    y_r = data_middle[ycol_highlight].max()

    traces.append(
        go.Scatter(
            x=[0, 0],
            y=[y_l, y_r],
            mode="markers",
            marker=dict(color="rgba(95,40,200,1)", size=6),
            showlegend=False,
            hovertemplate="<br><b>Cumulative Cost</b>: $%{y:,.0f}",
            name=ycol_highlight,
        )
    )

    traces.append(
        go.Scatter(
            x=[0],
            y=[(y_r + y_l) / 2],
            text=[f"${y_r - y_l:,.0f}"],
            textfont=dict(size=12, color="rgba(95,40,200,1)"),
            textposition="middle center",
            hoverinfo="skip",
            mode="text",
            showlegend=False,
        )
    )

    return traces

In [241]:
def plot_trend(data, xcol, columns_included, column_highlight, left, right):
    # subplots
    fig = make_subplots(
        rows=1,
        cols=2,
        column_widths=[8, 1],
        row_heights=[8],
        shared_yaxes=True,
        horizontal_spacing=0.02,
    )

    # plot lines in the main plot

    y_l = data.loc[data[xcol] == left, column_highlight].values[0]
    y_r = data.loc[data[xcol] == right, column_highlight].values[0]

    duration_sum = y_r - y_l

    main_traces = sum_lines_traces(
        data, columns_included, column_highlight, left, right, xcol
    )

    vert_traces = vertical_traces(data, column_highlight, left, right, xcol)

    for trace in main_traces:
        fig.add_trace(trace, row=1, col=1)

    for trace in vert_traces:
        fig.add_trace(trace, row=1, col=2)

    for y in [y_l, y_r]:
        fig.add_hline(y=y, line_dash="dot", row=1, col="all", opacity=0.5)

    # hide tick labels from the right plot
    fig.update_xaxes(
        range=[-0.1, 0.1],
        row=1,
        col=2,
        showgrid=False,  # This hides the vertical gridlines
        tickformat="%d-%b",  # This formats the tick labels (e.g., "01-Jan")
    )

    fig.update_layout(
        font=dict(size=14),
        width=800,
        height=600,
        title=f"Total Cost: {duration_sum:.2f}",
        # ymin
        yaxis=dict(
            range=[0, data[columns_included].max().max()],
            showgrid=False,
        ),
        xaxis=dict(
            showgrid=False,
        ),
        xaxis_title="Month",
        yaxis_title="Cumulative Cost ($)",
        hovermode="x",
    )

    # turn off grid
    for i in range(1, 3):
        fig.update_yaxes(showgrid=False, row=1, col=i)
        fig.update_xaxes(showgrid=False, row=1, col=i)

    fig.update_xaxes(showticklabels=False, row=1, col=2)

    fig.show()

In [242]:
plot_trend(data, "x", ["A", "B", "C"], "B", 6, 4 * 12 + 6)

## Old Code


In [ ]:
def add_vrect(fig, l, r, x_min, x_max):
    fig.add_vrect(
        x0=x_min,
        x1=l,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )
    fig.add_vrect(
        x0=r,
        x1=x_max,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )


def add_hrect(fig, b, t, y_min, y_max):
    fig.add_hrect(
        y0=y_min,
        y1=b,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )
    fig.add_hrect(
        y0=t,
        y1=y_max,
        fillcolor="black",
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )


def add_lines(fig, l, r, b, t):
    fig.add_vline(
        x=l, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )

    fig.add_vline(
        x=r, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )

    fig.add_hline(
        y=b, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )

    fig.add_hline(
        y=t, line_width=1, line_dash="dot", line_color="black", opacity=0.5
    )